In [1]:
# Criando um assistant tutor de tecnologia

Assistants:
permitem a construção de agentes específicos referentes a vários tipos de conhecimentos.

In [2]:
import openai
from dotenv import load_dotenv
import os

In [3]:
# Acessando API

load_dotenv()
secret_key = os.getenv("OPEN_API_KEY")

In [4]:
client = openai.Client(api_key=secret_key)

In [5]:
assistant = client.beta.assistants.create(
    name='Tutor de Tecnologia',
    instructions='Você é um tutor sobre assuntos relacionados a tecnologia',
    tools=[{'type':'code_interpreter'}],
    model='gpt-3.5-turbo-0125'
)

In [6]:
pergunta = """
Em um sistema de cache de mapeamento direto, dado uma memória principal de uma cache
de 256 KB e blocos e blocos de 64 bytes:
- Quantas linhas existem na cache?
- Onde o bloco de endereço 0x1A23C será mapeado na cache?
"""

In [7]:
# Criação da Thread

thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role='user',
    content= pergunta
)

In [8]:
# Executando a thread

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='Nome de usuário premium'
)

In [9]:
run

Run(id='run_UvBth9v2G3PdOLSjwPTj4jOZ', assistant_id='asst_xhSz0QsKhJm3TPuofye9LmlR', cancelled_at=None, completed_at=None, created_at=1745680859, expires_at=1745681459, failed_at=None, incomplete_details=None, instructions='Nome de usuário premium', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo-0125', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_8W2VjdPfie33S6KoNCSctZZL', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={}, reasoning_effort=None)

In [10]:
run.status

'queued'

queued -> estado de fila para ser executada.

In [11]:
# Aguardando a thread rodar

import time
while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run=client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [12]:
# Verificando a resposta

if run.status == 'completed':
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f'Erro{run.status}')

SyncCursorPage[Message](data=[Message(id='msg_XDumKYuAmqm6o1VrI1UlF63o', assistant_id='asst_xhSz0QsKhJm3TPuofye9LmlR', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O bloco de endereço 0x1A23C será mapeado na linha 572 da cache.\n\nPortanto, para um sistema de cache de mapeamento direto com uma memória cache de 256 KB e blocos de 64 bytes, existem 4096 linhas na cache e o bloco de endereço 0x1A23C será mapeado na linha 572 da cache.'), type='text')], created_at=1745680879, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_UvBth9v2G3PdOLSjwPTj4jOZ', status=None, thread_id='thread_8W2VjdPfie33S6KoNCSctZZL'), Message(id='msg_RlBriyGLmd5JM8uwAXgtZx4j', assistant_id='asst_xhSz0QsKhJm3TPuofye9LmlR', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Os bits de índice da cache para o endereço 0x1A23C são '001000111100'.\n\nPara determinar em

In [13]:
# Demonstrando a resposta

print(mensagens.data[0].content[0].text.value)

O bloco de endereço 0x1A23C será mapeado na linha 572 da cache.

Portanto, para um sistema de cache de mapeamento direto com uma memória cache de 256 KB e blocos de 64 bytes, existem 4096 linhas na cache e o bloco de endereço 0x1A23C será mapeado na linha 572 da cache.


In [14]:
# Analisando os passos do modelo

run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [15]:
# Passos que o modelo utilizou para resolver o problema

for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("="*10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        
        print(message.content[0].text.value)


====Step message_creation
Para calcular a quantidade de linhas na cache, primeiro precisamos determinar o tamanho de cada bloco em bits e, em seguida, podemos calcular o número de linhas na cache.

Dado que a cache possui 256 KB de tamanho e os blocos têm 64 bytes, podemos seguir os seguintes passos para encontrar as respostas:

1. Calcular o tamanho de cada bloco em bits.
2. Calcular o número de linhas na cache.
3. Determinar onde o bloco de endereço 0x1A23C será mapeado na cache.

Vamos começar calculando o número de linhas na cache:

====Step tool_calls
# Tamanho da cache em bytes
cache_size_kb = 256
cache_size_bytes = cache_size_kb * 1024

# Tamanho do bloco em bytes
block_size_bytes = 64

# Calculando o número de linhas na cache
num_lines_cache = cache_size_bytes // block_size_bytes
num_lines_cache

====Step message_creation
A cache de mapeamento direto tem 4096 linhas.

Agora vamos determinar onde o bloco de endereço 0x1A23C será mapeado na cache. Para isso, precisamos considera